In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
import holidays
import matplotlib.pyplot as plt
import seaborn as sns
import ydata_profiling


In [12]:
df = pd.read_parquet(Path("data") / "train.parquet")
weather_data = pd.read_csv("data/external_data.csv")

# Exploratory Data Analysis & Feature Engineering

## General EDA

In [13]:
df.info()
df.columns

<class 'pandas.core.frame.DataFrame'>
Index: 496827 entries, 48321 to 929187
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 496827 non-null  category      
 1   counter_name               496827 non-null  category      
 2   site_id                    496827 non-null  int64         
 3   site_name                  496827 non-null  category      
 4   bike_count                 496827 non-null  float64       
 5   date                       496827 non-null  datetime64[us]
 6   counter_installation_date  496827 non-null  datetime64[us]
 7   coordinates                496827 non-null  category      
 8   counter_technical_id       496827 non-null  category      
 9   latitude                   496827 non-null  float64       
 10  longitude                  496827 non-null  float64       
 11  log_bike_count             496827 non-null  float64  

Index(['counter_id', 'counter_name', 'site_id', 'site_name', 'bike_count',
       'date', 'counter_installation_date', 'coordinates',
       'counter_technical_id', 'latitude', 'longitude', 'log_bike_count'],
      dtype='object')

## DateTime related features

In [14]:
fr_holidays = holidays.France()

df["datetime"] = df["date"]
df["date"] = df["datetime"].dt.date
#df["hour"] = df["datetime"].dt.hour
df["weekday"] = df["datetime"].dt.weekday
df["daymonth"] = df["datetime"].dt.strftime('%d') + "_" + df["datetime"].dt.month.astype(str)
df["IsHoliday"] = df["datetime"].dt.date.apply(lambda x: x in fr_holidays)



In [15]:
df.groupby(["counter_id", "date"]).size().reset_index(name="count").sort_values(by="count", ascending=False)["count"].describe() # not always 24 records / day / counter

/var/folders/bn/bvffp5151hvf872qg5mg_l9w0000gn/T/ipykernel_28129/1829517537.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["counter_id", "date"]).size().reset_index(name="count").sort_values(by="count", ascending=False)["count"].describe() # not always 24 records / day / counter


count    20944.000000
mean        23.721686
std          2.542094
min          0.000000
25%         24.000000
50%         24.000000
75%         24.000000
max         24.000000
Name: count, dtype: float64

In [16]:
def add_lag_and_rolling_features(group):
    group = group.sort_values(by='datetime')
    
    group['lag_1'] = group['bike_count'].shift(1)
    group['lag_24'] = group['bike_count'].shift(24)
    group['lag_168'] = group['bike_count'].shift(168)
    
    group['rolling_mean_24h'] = group['bike_count'].rolling(window=24, min_periods=1).mean()
    group['rolling_std_24h'] = group['bike_count'].rolling(window=24, min_periods=1).std()
    group['rolling_mean_7d'] = group['bike_count'].rolling(window=168, min_periods=1).mean()
    group['rolling_std_7d'] = group['bike_count'].rolling(window=168, min_periods=1).std()

    return group

df_lag_rolling = df.groupby('counter_id').apply(add_lag_and_rolling_features).reset_index(drop=True)

display(df_lag_rolling.head())


/var/folders/bn/bvffp5151hvf872qg5mg_l9w0000gn/T/ipykernel_28129/3361494806.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_lag_rolling = df.groupby('counter_id').apply(add_lag_and_rolling_features).reset_index(drop=True)
/var/folders/bn/bvffp5151hvf872qg5mg_l9w0000gn/T/ipykernel_28129/3361494806.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_lag_rolling = df.groupby('counter_id').apply(add_lag_and_rolling_features).reset_index(drop=True)


,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,...,weekday,daymonth,IsHoliday,lag_1,lag_24,lag_168,rolling_mean_24h,rolling_std_24h,rolling_mean_7d,rolling_std_7d
0,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,1.0,2020-09-01,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,1,01_9,False,NaN,NaN,NaN,1.000000,NaN,1.000000,NaN
1,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,2.0,2020-09-01,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,1,01_9,False,1.0,NaN,NaN,1.500000,0.707107,1.500000,0.707107
2,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,7.0,2020-09-01,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,1,01_9,False,2.0,NaN,NaN,3.333333,3.214550,3.333333,3.214550
3,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,10.0,2020-09-01,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,1,01_9,False,7.0,NaN,NaN,5.000000,4.242641,5.000000,4.242641
4,100007049-101007049,28 boulevard Diderot O-E,100007049,28 boulevard Diderot,48.0,2020-09-01,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,1,01_9,False,10.0,NaN,NaN,13.600000,19.578049,13.600000,19.578049


In [17]:
print(df.shape)
print(df_lag_rolling.shape)
df_sorted = df.sort_values(by=df.columns[:7].tolist()).reset_index(drop=True)
df_lag_rolling_sorted = df_lag_rolling.sort_values(by=df_lag_rolling.columns[:7].tolist()).reset_index(drop=True)
print(df_sorted[df_sorted.columns[:7]].equals(df_lag_rolling_sorted[df_lag_rolling_sorted.columns[:7]]))

(496827, 16)
(496827, 23)
True


In [19]:
df_lag_rolling.columns

Index(['counter_id', 'counter_name', 'site_id', 'site_name', 'bike_count',
       'date', 'counter_installation_date', 'coordinates',
       'counter_technical_id', 'latitude', 'longitude', 'log_bike_count',
       'datetime', 'weekday', 'daymonth', 'IsHoliday', 'lag_1', 'lag_24',
       'lag_168', 'rolling_mean_24h', 'rolling_std_24h', 'rolling_mean_7d',
       'rolling_std_7d'],
      dtype='object')

In [18]:
df_lag_rolling.profile_report()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

AttributeError: 'datetime.date' object has no attribute 'hour'

## Location & weather related features